In [ ]:
import importlib
import ifera
import torch

importlib.reload(ifera)

cm = ifera.ConfigManager()
# instrument = config.get_config('IBKR', 'NIY:1m')

# ifera.check_s3_file_exists(S3_BUCKET, ifera.make_s3_key(instrument, True))

# df_raw = ifera.load_data(raw=True, instrument=instrument, zipfile=True)

# df = ifera.load_data(raw=False, instrument=instrument, zipfile=True, reset=True)

# instrument = config.get_config('IBKR', 'HE:1m')
# t_old = ifera.load_data_tensor(instrument=instrument, reset=False, device=torch.device('cuda:0'), dtype=torch.float32)
# t_new = ifera.load_data_tensor(instrument=instrument, reset=True, device=torch.device('cuda:0'), dtype=torch.float32)

# t_new.isclose(t_old).all()

# instrument = config.get_config('IBKR', 'NN:1m')
# t_nn = ifera.load_data_tensor(instrument=instrument, reset=False, device=torch.device('cuda:0'), dtype=torch.float32)

iconfig = cm.get_config('IBKR', 'GF:1m')
#t = ifera.load_data_tensor(instrument=instrument, reset=False, device=torch.device('cuda:0'), dtype=torch.float32)

instrument2 = cm.get_config('IBKR', 'GF:1m')
instrument3 = iconfig.model_copy()

iconfig.__eq__(instrument2)

In [ ]:


import pandas as pd

all_steps = pd.timedelta_range(start=pd.Timedelta(0), end=pd.Timedelta(days=1), freq=iconfig.time_step)
# Filter out steps that are greater than or equal to instrument end time
all_steps[all_steps < iconfig.trading_end - iconfig.trading_start][-1]


    


In [ ]:
import ifera

cm = ifera.ConfigManager()
brokerConfig = cm.get_broker_config('IBKR')

for key, baseConfig in cm.instruments_data.items():
    if baseConfig['symbol'] in brokerConfig.instruments.keys() and baseConfig['type'] == 'futures':
        iconfig = cm.get_config('IBKR', key)
        t = ifera.load_data_tensor(instrument=iconfig, reset=False, device=torch.device('cuda:0'), dtype=torch.float32)
    



In [ ]:
idata2 = ifera.InstrumentData(iconfig, device=torch.device('cuda:0'))

In [ ]:
ms = ifera.MarketSimulatorIntraday(idata2)

In [ ]:
import torch
import ifera
device=torch.device('cuda:0')

cm = ifera.ConfigManager()
iconfig = cm.get_config('IBKR', 'CL:1m')
dm = ifera.DataManager()
idata2 = dm.get_instrument_data(iconfig, device=device)
ms = ifera.MarketSimulatorIntraday(idata2)
t = (iconfig.liquid_start - iconfig.trading_start).total_seconds() // iconfig.time_step.total_seconds()

date_idx = torch.arange(0, idata2.data.shape[1], dtype=torch.int32, device=device)
time_idx = torch.full_like(date_idx, t, dtype=torch.int32, device=device)
position = torch.zeros_like(date_idx, dtype=torch.int32, device=device)
action = torch.full_like(date_idx, 1, dtype=torch.int32, device=device)
stop_loss = torch.full_like(date_idx, torch.nan, dtype=torch.float32, device=device)

%timeit ms.calculate_step(date_idx, time_idx, position, action, stop_loss)
#idata.data[0, 0:10]

In [ ]:
from einops import rearrange, repeat

parent_data = idata2.data
multiplier = 120
parent_steps = parent_data.shape[1]
padding = parent_steps - parent_steps // multiplier * multiplier
padding_data = torch.zeros((parent_data.shape[0], parent_data.shape[2]), dtype=parent_data.dtype, device=parent_data.device)
padding_data[:, 0:4] = parent_data[:, -1, 3, None]  # Open, High, Low, Close <- Last Close
padding_data = repeat(padding_data, "d c -> d g c", g=padding)
parent_data = torch.cat([parent_data, padding_data], dim=1)
rearrange(parent_data, "d (g n) c -> d g c n", n=120).shape

In [ ]:
idata2.instrument.contract_multiplier * (1051.0 - idata2.instrument.min_slippage - 1051.0) - 2.25, idata2.instrument.min_slippage * idata2.instrument.contract_multiplier

In [ ]:
import datetime as dt

iconfig = cm.get_config('IBKR', 'CL:1m')
t = ifera.load_data_tensor(instrument=iconfig, reset=False, device=torch.device('cuda:0'), dtype=torch.float32)
t[-60 * 8 + 30], dt.date.fromordinal(int(t[-60 * 8 + 30, 0].item())), dt.timedelta(seconds=t[-60 * 8 + 30, 1].item())

In [ ]:
df = ifera.load_data(raw=True, instrument=iconfig, zipfile=True)

In [ ]:
# df[dt.datetime(2025, 1, 13, 9, 30)]
import pandas as pd

iconfig = cm.get_config('IBKR', 'NIY:1m')

df = ifera.load_data(raw=True, instrument=iconfig, zipfile=True)
t = ifera.load_data_tensor(instrument=iconfig, reset=False, device=torch.device('cuda:0'), dtype=torch.float32)

print(df.loc[dt.datetime(2025, 1, 10, 9, 30)])
idx = -60 * 8 + 30 - 23 * 60 
s = dt.timedelta(seconds=t[idx, 1].item()).seconds
f"Date: {dt.date.fromordinal(int(t[idx, 0].item()))}, Time: {s//3600}:{(s%3600)//60}:{s%60:02d}, Open: {t[idx, 4].item():.2f}, High: {t[idx, 5].item():.2f}, Low: {t[idx, 6].item():.2f}, Close: {t[idx, 7].item():.2f}, Volume: {t[idx, 8].item():.0f}"



In [ ]:
df.tail()
df.loc[dt.datetime(2025, 1, 10, 9, 30)]

In [ ]:
import boto3

s3 = boto3.client('s3')

response = s3.list_objects(Bucket="kibotdata", Prefix='futures/1m/')
for obj in sorted(response.get('Contents', []), key=lambda x: x['Size'], reverse=True):
    print(obj['Key'], obj['Size'] // 1024**2)


In [ ]:
import ifera

cm = ifera.ConfigManager()
broker = cm.get_broker_config('IBKR')

for iconfig in broker.instruments:
    iconfig = cm.get_config('IBKR', f"{iconfig}:1m")
    df = ifera.load_data(raw=False, instrument=iconfig, zipfile=True)



In [ ]:
import ifera
import torch
from einops import rearrange

cm = ifera.ConfigManager()
broker = cm.get_broker_config('IBKR')

for key in cm.instruments_data.keys():
    base_inst = cm.get_base_instrument_config(key)
    if base_inst.symbol not in broker.instruments:
        continue
    iconfig = cm.get_config('IBKR', key)
    t = ifera.load_data_tensor(instrument=iconfig, reset=False, device=torch.device('cuda:0'), dtype=torch.float32)
    t = rearrange(t, '(d t) c -> d t c', t = iconfig.total_steps)
    print(iconfig.symbol, t.shape)
    break

In [ ]:
data = t[:, :, 4:]

In [ ]:
import torch
import ifera
channels = {"Open":0, "High":1, "Low":2, "Close":3, "Volume":4}

# Relative True Range: max(high, prev_close) / min(low, prev_close), and simple high/low ratio for the first bar
rtr = torch.zeros_like(data[:, :, channels["Close"]])
rtr[:, 0] = data[:, 0, channels["High"]] / data[:, 0, channels["Low"]] - 1.0
rtr[:, 1:] = torch.max(data[:, 1:, channels["High"]], data[:, :-1, channels["Close"]]) / torch.min(data[:, 1:, channels["Low"]], data[:, :-1, channels["Close"]]) - 1.0

# Where volume is zero set rtr to be the same as the previous value

In [ ]:
from einops import rearrange, repeat

channels = {"Open":0, "High":1, "Low":2, "Close":3, "Volume":4}

volume = data[:, :, channels["Volume"]].to(torch.int32)

# Find first non-zero volume each day
first_non_zero = torch.argmax((volume > 0).to(torch.int8), dim=1)

rtr = torch.zeros_like(data[:, :, channels["Close"]])
rtr[:, first_non_zero] = data[:, first_non_zero, channels["High"]] / data[:, first_non_zero, channels["Low"]] - 1.0

mask = volume != 0
vdata = data[mask, :]
vrtr = rtr[mask]

raw_rtr = torch.max(vdata[1:, channels["High"]], vdata[:-1, channels["Close"]]) / torch.min(vdata[1:, channels["Low"]], vdata[:-1, channels["Close"]]) - 1.0
vrtr[1:] = torch.where(vrtr[1:] == 0, raw_rtr, vrtr[1:])

artr = ifera.ema_slow(vrtr, 1/14)

artr

In [ ]:
import torch

t = torch.tensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [11, 12, 13, 14, 15, 16, 17, 18, 19, 20]], dtype=torch.float32)
window = 3

n = min(t.shape[-1], window)
tail = t.unfold(dimension=-1, size=n, step=1).mean(dim=-1)

head_nom = torch.cumsum(t[..., :n-1], dim=-1)
head_denom = torch.arange(1, n, device=t.device, dtype=t.dtype)
head = head_nom / head_denom

sma = torch.cat([head, tail], dim=-1)
sma


In [ ]:
import torch
import ifera
from einops import rearrange

cm = ifera.ConfigManager()
iconfig = cm.get_config("IBKR", "CL:1m")

dm = ifera.DataManager()
idata = dm.get_instrument_data(iconfig)

batch_size = idata.data.shape[0]-250

openPolicy = ifera.AlwaysOpenPolicy(direction=1)
initStopPolicy = ifera.InitialArtrStopLossPolicy(
    instrument_data=idata, atr_multiple=3.0
)
maintenancePolicy = ifera.ScaledArtrMaintenancePolicy(
    instrument_data=idata,
    stages=["1m", "5m", "15m", "1h", "4h", "1d"],
    atr_multiple=3.0,
    wait_for_breakeven=False,
    minimum_improvement=0.0,
)

tradingPolicy = ifera.TradingPolicy(
    instrument_data=idata,
    open_position_policy=openPolicy,
    initial_stop_loss_policy=initStopPolicy,
    position_maintenance_policy=maintenancePolicy,
)

#tradingPolicy = torch.compile(tradingPolicy)

ms = ifera.MarketSimulatorIntraday(instrument_data=idata)

t = (iconfig.liquid_start - iconfig.trading_start).total_seconds() // iconfig.time_step.total_seconds() - 1

date_idx = torch.arange(0, batch_size, dtype=torch.int32, device=idata.device)
time_idx = torch.full_like(date_idx, t, dtype=torch.int32, device=idata.device)
position = torch.zeros_like(date_idx, dtype=torch.int32, device=idata.device)
prev_stop_loss = torch.full_like(date_idx, torch.nan, dtype=torch.float32, device=idata.device)
execution_price = torch.full_like(date_idx, torch.nan, dtype=torch.float32, device=idata.device)

total_profit = torch.zeros_like(date_idx, dtype=torch.float32, device=idata.device)

# Open position
action, stop_loss = tradingPolicy(date_idx, time_idx, position, prev_stop_loss, execution_price)
time_idx += 1
profit, position, execution_price, _ = ms.calculate_step(date_idx, time_idx, position, action, stop_loss)
position = position.clone()
total_profit += profit

while (position != 0).any():
    prev_stop_loss = stop_loss.clone()
    action, stop_loss = tradingPolicy(date_idx, time_idx, position, prev_stop_loss, execution_price)
    date_idx, time_idx = idata.get_next_indices(date_idx, time_idx)
    action = torch.where((date_idx == idata.data.shape[0] - 1) & (time_idx == idata.data.shape[1] - 1), -position, 0)
    profit, position, execution_price, _ = ms.calculate_step(date_idx, time_idx, position, action, stop_loss)
    position = position.clone()
    total_profit += profit

max_idx = total_profit.argmax()
print(f"Max index: {max_idx}, Total profit: {total_profit[max_idx].item():.4f}, date_idx: {date_idx[max_idx].item()}, time_idx: {time_idx[max_idx].item()}")


In [ ]:
import torch
import numpy as np

t = torch.arange(0, 10, dtype=torch.int32)
mask = torch.zeros_like(t, dtype=torch.bool)

t, mask, torch.cat((t[mask], torch.tensor([np.iinfo(np.int32).max], dtype=torch.int32)), dim=0).min().item(), t[t< 5]



In [ ]:
import ifera
import torch

instrument = ifera.ConfigManager().get_base_instrument_config("TN:1m")
url = ifera.make_instrument_url(ifera.Scheme.FILE, ifera.Source.PROCESSED, instrument)
# print(url)

#t = ifera.load_data_tensor(instrument=instrument, reset=False, device=torch.device('cuda:0'), dtype=torch.float32)
fm = ifera.FileManager()
fm.refresh_file("adasd")

# t.shape
# print(ifera.FileOperations.exists(url))

In [ ]:
import ifera
import torch

dm = ifera.DataManager()
cm = ifera.ConfigManager()
#parent_instrument = cm.get_base_instrument_config("TN", "1m")
#instrument = cm.create_derived_base_config(parent_instrument, "5m")
instrument = cm.get_base_instrument_config("TN", "30m")
device=torch.device("cuda:0")

idata = dm.get_instrument_data(instrument_config=instrument, dtype=torch.float32, device=device)

idata.data.shape


In [ ]:
import torch
import ifera
import time
from tqdm import tqdm

dm = ifera.DataManager()
cm = ifera.ConfigManager()
instrument = cm.get_base_instrument_config("TN:1m")
idata = dm.get_instrument_data(instrument_config=instrument, dtype=torch.float32, device=torch.device("cuda:0"))
print(idata.data.shape)
ms = ifera.MarketSimulatorIntraday(instrument_data=idata, broker_name="IBKR")

date_idx = torch.zeros(idata.data.shape[0], dtype=torch.int32, device=idata.device)
time_idx = torch.full_like(date_idx, 0, dtype=torch.int32, device=idata.device)
position = torch.full_like(date_idx, 1, dtype=torch.int32, device=idata.device)
action = torch.full_like(date_idx, 0, dtype=torch.int32, device=idata.device)
stop_loss = torch.full_like(date_idx, torch.nan, dtype=torch.float32, device=idata.device)

def run_market_simulation(ms, idata):
    date_idx = torch.randint(0, idata.data.shape[0] - total_days - 2, (idata.data.shape[0],), dtype=torch.int32, device=idata.device)
    time_idx = torch.randint_like(date_idx, 0, idata.data.shape[1] - 1, dtype=torch.int32, device=idata.device)
    position = torch.full_like(date_idx, 0, dtype=torch.int32, device=idata.device)
    action = torch.full_like(date_idx, 1, dtype=torch.int32, device=idata.device)
    stop_loss = torch.full_like(date_idx, torch.nan, dtype=torch.float32, device=idata.device)
    total_profit = torch.zeros_like(date_idx, dtype=torch.float32, device=idata.device)
    steps = torch.tensor(0, dtype=torch.int32, device=idata.device)
    max_days = 4

    while date_idx[0] < max_days or time_idx[0] < idata.data.shape[1] - 1:
        profit, new_position, _, _ = ms.calculate_step(date_idx, time_idx, position, action, stop_loss)
        position = new_position.clone()
        action = torch.zeros_like(date_idx, dtype=torch.int32, device=idata.device)
        date_idx, time_idx = idata.get_next_indices(date_idx, time_idx)
        total_profit += profit
        steps += 1
    
    return total_profit, steps

def run_market_simulation_tqdm(ms, idata):
    total_days = 5
    date_idx = torch.randint(0, idata.data.shape[0] - total_days - 2, (idata.data.shape[0],), dtype=torch.int32, device=idata.device)
    time_idx = torch.randint_like(date_idx, 0, idata.data.shape[1] - 1, dtype=torch.int32, device=idata.device)
    position = torch.full_like(date_idx, 0, dtype=torch.int32, device=idata.device)
    action = torch.full_like(date_idx, 1, dtype=torch.int32, device=idata.device)
    stop_loss = torch.full_like(date_idx, torch.nan, dtype=torch.float32, device=idata.device)
    total_profit = torch.zeros_like(date_idx, dtype=torch.float32, device=idata.device)
    steps = torch.tensor(0, dtype=torch.int32, device=idata.device)
    total_steps = total_days * idata.data.shape[1]
    pbar = tqdm(total=total_steps, desc="Market Simulation", unit="step", leave=False)

    while steps < total_steps:
        profit, new_position, _, _ = ms.calculate_step(date_idx, time_idx, position, action, stop_loss)
        total_profit += profit
        position = new_position.clone()
        action = torch.where(position == 0, action, 0)
        date_idx, time_idx = idata.get_next_indices(date_idx, time_idx)
        steps += 1
        pbar.update(1)
    
    return total_profit, steps


run_market_simulation_compiled = torch.compile(run_market_simulation)
run_market_simulation_tqdm_compiled = torch.compile(run_market_simulation_tqdm)

# Warmup calculate_step
profit, _, _, _ = ms.calculate_step(date_idx, time_idx, position, action, stop_loss)
profit, _, _, _ = ms.calculate_step(date_idx, time_idx, position, action, stop_loss)
profit, _, _, _ = ms.calculate_step(date_idx, time_idx, position, action, stop_loss)

# Run the market simulation
start_t = time.time()
total_profit, steps = run_market_simulation(ms, idata)
print(f"Time taken: {time.time() - start_t:.2f} seconds")

# Warmup the compiled function
total_profit, steps = run_market_simulation_compiled(ms, idata)
total_profit, steps = run_market_simulation_compiled(ms, idata)
start_t = time.time()
total_profit, steps = run_market_simulation_compiled(ms, idata)
print(f"Time taken: {time.time() - start_t:.2f} seconds")

# Run uncompiled function with tqdm
start_t = time.time()
total_profit, steps = run_market_simulation_tqdm(ms, idata)
print(f"Time taken: {time.time() - start_t:.2f} seconds")

# Warmup the compiled function with tqdm
total_profit, steps = run_market_simulation_tqdm_compiled(ms, idata)
total_profit, steps = run_market_simulation_tqdm_compiled(ms, idata)
start_t = time.time()
total_profit, steps = run_market_simulation_tqdm_compiled(ms, idata)
print(f"Time taken: {time.time() - start_t:.2f} seconds")
print(total_profit)



In [ ]:
import boto3
import botocore.exceptions

client = boto3.client("s3")
bucket = "ifera-marketdata"

try:
    client.head_object(Bucket=bucket, Key='asdasdasd')
except botocore.exceptions.ClientError as e:
    print(e.response['Error'])


In [ ]:
from datetime import datetime, timedelta

timedelta(hours=16,minutes=24,seconds=47) / timedelta(minutes=30,seconds=5)


In [ ]:
import torch
from torch._higher_order_ops.foreach_map import foreach_map

class TreeNode:
    """
    A node in the decision tree.
    """
    __slots__ = ("is_leaf", "prediction", "feature_index", "threshold", "left", "right", "depth")

    def __init__(self, is_leaf=False, prediction=None, feature_index=None, threshold=None, left=None, right=None, depth=0):
        self.is_leaf = is_leaf
        self.prediction = prediction
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.depth = depth


class DecisionTreeRegressorTorch:
    """
    A regression decision tree implemented in PyTorch with iterative breadth-first build.

    Splits nodes to minimize mean squared error (MSE).
    Supports max_depth and a minimum impurity decrease threshold.
    Uses PyTorch 2.7 torch._foreach_map for parallel split search at each depth.
    """

    def __init__(self, max_depth: int = 5, min_impurity_decrease: float = 1e-7):
        self.max_depth = max_depth
        self.min_impurity_decrease = min_impurity_decrease
        self.root = None

    #@torch.compile
    def fit(self, X: torch.Tensor, y: torch.Tensor):
        """
        Build the tree by iterating over depth levels.
        """
        assert X.dim() == 2 and y.dim() == 1 and X.size(0) == y.size(0)

        # Initialize root node
        self.root = TreeNode()
        # Track nodes, inputs, targets across depths
        nodes = [self.root]
        inputs = [X]
        targets = [y]

        for depth in range(self.max_depth):
            next_nodes = []
            next_inputs = []
            next_targets = []

            # Compute parent variances in parallel using foreach_map
            
            parent_vars = foreach_map(torch.var, targets, correction=0)
            # Find best splits for all nodes at this depth in parallel
            splits = foreach_map(self._find_best_split, inputs, targets, parent_vars)

            for node, X_sub, y_sub, split in zip(nodes, inputs, targets, splits):
                feature, threshold, imp_dec = split
                n_samples = y_sub.numel()
                node.depth = depth
                # Stop splitting if no valid split, too few samples, or insufficient decrease
                if feature is None or n_samples < 2 or imp_dec < self.min_impurity_decrease:
                    node.is_leaf = True
                    node.prediction = y_sub.mean().item()
                else:
                    node.is_leaf = False
                    node.feature_index = feature
                    node.threshold = threshold

                    # Partition data
                    mask = X_sub[:, feature] <= threshold
                    X_left, y_left = X_sub[mask], y_sub[mask]
                    X_right, y_right = X_sub[~mask], y_sub[~mask]

                    node.left = TreeNode()
                    node.right = TreeNode()

                    # Queue children for next level
                    next_nodes.extend([node.left, node.right])
                    next_inputs.extend([X_left, X_right])
                    next_targets.extend([y_left, y_right])

            # Move to next depth
            nodes, inputs, targets = next_nodes, next_inputs, next_targets
            if len(nodes) == 0:
                break

        # Any remaining nodes become leaves
        for node, y_sub in zip(nodes, targets):
            node.is_leaf = True
            node.prediction = y_sub.mean().item()

    def predict(self, X: torch.Tensor) -> torch.Tensor:
        """
        Predict target values for input X.
        """
        preds = []
        for x in X:
            node = self.root
            while not node.is_leaf:
                node = node.left if x[node.feature_index] <= node.threshold else node.right
            preds.append(node.prediction)
        return torch.tensor(preds, dtype=X.dtype)

    def _find_best_split(self, X: torch.Tensor, y: torch.Tensor, parent_variance: float):  # noqa: C901
        """
        Vectorized single-node split search used by foreach_map.
        Returns (feature_index, threshold, impurity_decrease).
        """
        n_samples, n_features = X.shape
        if n_samples < 2:
            return None, None, 0.0

        # Sort features and align targets
        sorted_vals, sorted_idx = torch.sort(X, dim=0)
        y_sorted = y[sorted_idx]

        # Cumulative sums
        csum_y = torch.cumsum(y_sorted, dim=0)
        csum_y2 = torch.cumsum(y_sorted * y_sorted, dim=0)

        # Split counts
        left_counts = torch.arange(1, n_samples, device=X.device).unsqueeze(1).expand(n_samples-1, n_features)
        right_counts = n_samples - left_counts

        # Sums and sums of squares
        left_sum = csum_y[:-1]
        left_sum2 = csum_y2[:-1]
        y_tot = y.sum()
        y2_tot = (y * y).sum()
        right_sum = y_tot - left_sum
        right_sum2 = y2_tot - left_sum2

        # Compute variances
        left_var = left_sum2 / left_counts - (left_sum / left_counts) ** 2
        right_var = right_sum2 / right_counts - (right_sum / right_counts) ** 2

        # Weighted child variance
        child_var = (left_counts * left_var + right_counts * right_var) / n_samples

        # Impurity decrease
        imp_dec = parent_variance - child_var
        distinct = sorted_vals[:-1] != sorted_vals[1:]
        imp_dec = imp_dec.masked_fill(~distinct, float('-inf'))

        # Select best split
        max_val, idx = imp_dec.reshape(-1).max(dim=0)
        if max_val <= 0:
            return None, None, 0.0

        feat = idx % n_features
        pos = idx // n_features
        thresh = (sorted_vals[pos, feat] + sorted_vals[pos+1, feat]) / 2.0
        return feat.item(), thresh.item(), max_val.item()

    def get_leaf_nodes(self) -> list:
        """
        Return all leaf nodes in the tree.
        """
        leaves = []
        stack = [self.root]
        while stack:
            node = stack.pop()
            if node.is_leaf:
                leaves.append(node)
            else:
                # push children to stack
                if node.right is not None:
                    stack.append(node.right)
                if node.left is not None:
                    stack.append(node.left)
        return leaves

    def get_leaves_sorted(self) -> list:
        """
        Return leaf nodes sorted by their prediction values (ascending).
        """
        leaves = self.get_leaf_nodes()
        return sorted(leaves, key=lambda n: n.prediction)


In [ ]:
import numpy as np

X = torch.rand(1000, 30, device="cuda:0")
y = torch.rand(1000, device="cuda:0")
tree = DecisionTreeRegressorTorch(max_depth=1000000, min_impurity_decrease=1e-5)
tree.fit(X, y)
leaves = tree.get_leaves_sorted()

len(leaves), np.max([n.depth for n in leaves])

In [ ]:
import ifera
import torch
from torch._higher_order_ops.foreach_map import foreach_map


fm = ifera.FileManager()
url = "file:data/tensor/futures_rollover/30m/GC.pt"
fm.build_subgraph(url, ifera.RuleType.REFRESH)
cm = ifera.ConfigManager()
base_instrument = cm.get_base_instrument_config("GC","30m")
base_tensor = torch.tensor([])
contract_instruments = []
contract_tensors = []

get_params_time = 0
get_tensor_time = 0

for dep in fm.refresh_graph.successors(url):
    fm.refresh_file(dep)
    params = fm.get_node_params(ifera.RuleType.REFRESH, dep)

    if params["type"] == "futures":
        base_tensor = ifera.load_data_tensor(
            instrument=base_instrument,
            reset=False,
            dtype=torch.float32,
            device=torch.device("cuda:0"),
            strip_date_time=False,
        )
    else:
        contract_instrument = cm.create_derived_base_config(
            base_instrument,
            contract_code=params["contract_code"],
        )
        contract_tensor = ifera.load_data_tensor(
            instrument=contract_instrument,
            reset=False,
            dtype=torch.float32,
            device=torch.device("cuda:0"),
            strip_date_time=False,
        )
        contract_tensors.append(contract_tensor)
        contract_instruments.append(contract_instrument)


Loading data from data/raw/futures_individual/30m/GC-F11.zip: 100%|██████████| 1629/1629 [00:00<00:00, 211579.02lines/s]


Converting datetime columns...
Processing groups...


Processing trade dates: 100%|██████████| 64/64 [00:00<00:00, 165.27it/s]


Performing final calculations...
Saving processed data...
Processed data saved to data/processed/futures_individual/30m/GC-F11.zip


Uploading processed/futures_individual/30m/GC-F11.zip: 100%|██████████| 30.2k/30.2k [00:00<00:00, 37.2kB/s]
Loading data from data/processed/futures_individual/30m/GC-F11.zip: 100%|██████████| 2944/2944 [00:00<00:00, 315588.49lines/s]
Uploading tensor/futures_individual/30m/GC-F11.pt: 100%|██████████| 107k/107k [00:00<00:00, 112kB/s]

In [ ]:
import numpy as np
import datetime as dt


agg_contract_tensor = torch.zeros(
    (len(contract_tensors), base_tensor.shape[0], base_tensor.shape[1], base_tensor.shape[2]), 
    dtype=base_tensor.dtype,
    device=base_tensor.device,
)

contract_tensors = sorted(contract_tensors, key=lambda x: (x[-1, 0, 2].to(torch.int64).item()))



In [ ]:
def copy_contract_tensors_to_agg(contract_tensors, base_tensor, agg_contract_tensor):
    """
    Copies data from contract_tensors to agg_contract_tensor based on matching dates.
    
    Args:
        contract_tensors (list): List of tensors, each shape (date_i, time, features)
        base_tensor (torch.Tensor): Shape (date_base, time, features)
        agg_contract_tensor (torch.Tensor): Shape (n, date_base, time, features)
    """
    # Extract dates from base_tensor, shape (date_base,)
    base_dates = base_tensor[:, 0, 2]
    
    # Compute matching indices for each contract_tensor
    b_indices_list = []
    for i in range(len(contract_tensors)):
        # Extract dates, shape (date_i,)
        contract_dates = contract_tensors[i][:, 0, 2]
        # Broadcasting to find matches, shape (date_i, date_base)
        matches = contract_dates[:, None] == base_dates[None, :]
        # Get base indices, shape (date_i,)
        b_indices = matches.long().argmax(dim=1)
        b_indices_list.append(b_indices)
    
    # Concatenate all base indices, shape (total_dates,)
    all_b_idx = torch.cat(b_indices_list, dim=0)
    
    # Create tensor of contract indices, shape (total_dates,)
    all_i = torch.cat([
        torch.full((contract_tensors[i].shape[0],), i, dtype=torch.long)
        for i in range(len(contract_tensors))
    ], dim=0)
    
    # Concatenate all contract data, shape (total_dates, time, features)
    all_data = torch.cat(contract_tensors, dim=0)
    
    # Assign data to agg_contract_tensor in one operation
    agg_contract_tensor[all_i, all_b_idx, :, :] = all_data

In [ ]:
copy_contract_tensors_to_agg(contract_tensors, base_tensor, agg_contract_tensor)
agg_contract_tensor.shape

In [ ]:
import numpy as np
import datetime as dt

agg_index = (agg_contract_tensor[0, :, 0, 2] == contract_tensors[0][21, 0, 2]).to(torch.int64).argmax()

(agg_contract_tensor[:, :, 0, 2].count_nonzero(0) == 0).to(torch.int64).argmax()

orddates = base_tensor[:, 0, 2].to(torch.int64).to("cpu").numpy()
dates = np.array([dt.date.fromordinal(int(x)) for x in orddates])
weekends = np.array([dt.date.weekday(x) for x in dates]) >= 5
weekends.sum()

base_tensor[623, :, :]

#TODO: Remove 0 volume dates (weekends)



# Get the indices in dim=0 where agg_contract_tensor[:, 0, 0, 2] is not 0
idx = agg_contract_tensor[:, 0, 0, 2].nonzero(as_tuple=True)[0]

print(idx)

print(np.array2string(base_tensor[0, 0, 4:].cpu().numpy(),
    formatter={'float_kind': lambda x: f"{x:.2f}"}))
    
print(np.array2string(agg_contract_tensor[idx, 0, 0, 4:].cpu().numpy(),
    formatter={'float_kind': lambda x: f"{x:.2f}"}))

In [ ]:
base_features = base_tensor[:, :, 4:8].clone()
agg_features = agg_contract_tensor[:, :, :, 4:8]

base_features = torch.where(base_tensor[:, :, 8].unsqueeze(-1) == 0, torch.nan, base_features)

matches = torch.all(
        agg_features == base_features[None, :, :, :],
        dim=-1
    )

result = torch.full(
    (base_tensor.shape[0], base_tensor.shape[1]),
    float('nan'),
    dtype=torch.float,
    device=base_tensor.device
)

any_match = matches.any(dim=0)
first_match_idx = matches.long().argmax(dim=0)
result[any_match] = first_match_idx[any_match].float()


d = 24
t = 44

print(result[d])
print(np.array2string(result.nan_to_num(-1).max(dim=1)[0][0:200].to(torch.int64).cpu().numpy(), 
      formatter={'float_kind': lambda x: f"{x:.2f}"}))

base_tensor[d, t, 4:9], agg_contract_tensor[0, d, t, 4:8], agg_contract_tensor[1, d, t, 4:8], agg_contract_tensor[2, d, t, 4:8], agg_contract_tensor[3, d, t, 4:8], agg_contract_tensor[14, d, t, 4:9], \
    dt.date.fromordinal(base_tensor[d, t, 2].to(torch.int64).item()),\
    dt.date.fromordinal(contract_tensors[0][-1, t, 2].to(torch.int64).item())
    


